In [ ]:
!pip3 show openai

In [1]:
!pip install opencv-python-headless
!pip install google-cloud-vision

# Import libraries and dataset

In [1]:
import pandas as pd
import os
import io
import json
from google.cloud import vision
from google.oauth2 import service_account
import requests

train = pd.read_csv("llm/data/LLM-TRAIN.csv")
test = pd.read_csv("llm/data/LLM-TEST.csv")



# Call Google Vision API to extract text from images

In [8]:

# Directory containing invoice files
directory = "llm/data/Test Files"

# Dictionary to store the results
ocr_results = {}


# Path to your Google Cloud service account key file
service_account_key_path = 'primal-oxide-406911-30736c9493e4.json'
credentials = service_account.Credentials.from_service_account_file(service_account_key_path)

# Initialize the Google Cloud Vision client
client = vision.ImageAnnotatorClient(credentials=credentials)

def detect_text_and_boxes(image_path):
    """Detects text and their bounding boxes in the file."""
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = client.text_detection(image=image)

    if response.error.message:
        raise Exception(f'Google Cloud Vision API error: {response.error.message}')

    results = []
    for text in response.text_annotations[1:]:  # Skip the first element which is the entire block of text
        vertices = text.bounding_poly.vertices
        # Top-left (x_min, y_min) and bottom-right (x_max, y_max) vertices
        bbox = [vertices[0].x, vertices[0].y, vertices[2].x, vertices[2].y]
        results.append({'text': text.description, 'bbox': bbox})
    return results


# Process each file in the directory
for filename in os.listdir(directory):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(directory, filename)
        try:
            ocr_results[filename] = detect_text_and_boxes(file_path)
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

# Write the results to a JSON file
with open('ocr_llm_results_test.json', 'w') as json_file:
    json.dump(ocr_results, json_file, indent=4)


In [3]:
# import json

# # Path to the JSON file
# json_file_path = 'ocr_llm_results.json'

# # Load the JSON file
# with open(json_file_path, 'r') as file:
#     ocr_results = json.load(file)

# Creating prompt text

In [10]:
# Function to preprocess OCR data for GPT
def preprocess_for_gpt(ocr_data):
    # Combining text entries and minimizing token usage
    combined_text = "text, [bbox] \n "
    for item in ocr_data:
        combined_text += f"{item['text']}, {item['bbox']} \n "

    return combined_text.strip()


In [11]:
def generate_key_details_prompt(ocr_data):
    # Read the static part of the prompt from the file
    with open('./prompt.txt', 'r') as file:
        prompt = file.read()

    # Insert the OCR data into the prompt at the specified location
    prompt = prompt + "\n " + preprocess_for_gpt(ocr_data)
    
    return prompt

# Calling OpenAI API

In [12]:
import os
from openai import OpenAI
from tqdm import tqdm


responses = []

my_client = OpenAI(api_key="sk-xxx")
# count = 0

# Loop through each OCR result and generate chat completions
for key, ocr_data in tqdm(ocr_results.items(), desc="Processing OCR data"):
    # # toremove
    # if count >=2:
    #     break
    
    # Format the OCR data as a string
    message = generate_key_details_prompt(ocr_data)
    
    # Create the chat completion request
    chat_completion = my_client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant skilled in converting OCR data into a structured JSON format."
            },
            {
                "role": "user",
                "content": message
            }
        ]
    )

    # Extract the response and add it to the responses list
    assistant_message = chat_completion.choices[0].message
    responses.append({"key": key, "response": assistant_message.content})
    
    # toremove
    # count += 1


# Print or process the responses as needed
for response in responses:
    print(f"Key: {response['key']}")
    print(f"JSON Response: {response['response']}\n")


Processing OCR data:   0%|          | 0/41 [00:00<?, ?it/s]

Processing OCR data: 100%|██████████| 41/41 [07:16<00:00, 10.65s/it]

Key: 01AR_print_inv.jpg
JSON Response: ```json
{
  "invoice_number": "30037",
  "invoice_number_bbox": "[1492, 177, 1570, 197]",
  "issue_date": "DEC03",
  "issue_date_bbox": "[1487, 242, 1572, 263]",
  "total": "8,385.00",
  "total_bbox": "[882, 1809, 962, 1828]",
  "table": "Item Description Qty. Unit Price Ext. Price\nAEROCHAIR - E1 Aero Executive Tile Chair 5 $500.00 $2,500.00\nCHAIR - SW4 Office Chair - Swivel Grey 4 Units $478.00 $1,912.00\nCHAIR - SW4 Office Chair - Swivel Grey 5 $699.00 $3,495.00",
  "table_bbox": "[451, 960, 1670, 1730]"
}
```

Key: creating-invoices-on-excel.png
JSON Response: Based on the provided OCR data, here is the structured JSON output extracting the relevant invoice information:

```json
{
  "invoice_number": "1000001",
  "invoice_number_bbox": "[772, 198, 824, 210]",
  "issue_date": "July 6, 2012",
  "issue_date_bbox": "[771, 175, 854, 189]",
  "total": "$2,407.00",
  "total_bbox": "[897, 1002, 955, 1014]",
  "table": "QUANTITY DESCRIPTION UNIT PRICE

In [13]:
responses

[{'key': '01AR_print_inv.jpg',
  'response': '```json\n{\n  "invoice_number": "30037",\n  "invoice_number_bbox": "[1492, 177, 1570, 197]",\n  "issue_date": "DEC03",\n  "issue_date_bbox": "[1487, 242, 1572, 263]",\n  "total": "8,385.00",\n  "total_bbox": "[882, 1809, 962, 1828]",\n  "table": "Item Description Qty. Unit Price Ext. Price\\nAEROCHAIR - E1 Aero Executive Tile Chair 5 $500.00 $2,500.00\\nCHAIR - SW4 Office Chair - Swivel Grey 4 Units $478.00 $1,912.00\\nCHAIR - SW4 Office Chair - Swivel Grey 5 $699.00 $3,495.00",\n  "table_bbox": "[451, 960, 1670, 1730]"\n}\n```'},
 {'key': 'creating-invoices-on-excel.png',
  'response': 'Based on the provided OCR data, here is the structured JSON output extracting the relevant invoice information:\n\n```json\n{\n  "invoice_number": "1000001",\n  "invoice_number_bbox": "[772, 198, 824, 210]",\n  "issue_date": "July 6, 2012",\n  "issue_date_bbox": "[771, 175, 854, 189]",\n  "total": "$2,407.00",\n  "total_bbox": "[897, 1002, 955, 1014]",\n  "

# Annotating the image

In [61]:
# For the GOogle OCR Results

# from PIL import Image, ImageDraw

# def annotate_image(image_path, annotations):
#     # Open the image
#     image = Image.open(image_path)
#     draw = ImageDraw.Draw(image)

#     for item in annotations:
#         text = item['text']
#         bbox = item['bbox']

#         # Draw the bounding box
#         draw.rectangle([bbox[0], bbox[1], bbox[2], bbox[3]], outline='red', width=2)

#         # Draw the text slightly above the top-left corner of the bounding box
#         draw.text((bbox[0], bbox[1] - 15), text, fill='black')  # Adjust y-offset as needed

#     return image

# # Annotate each image and save
# for filename, data in ocr_results.items():
#     image_path = os.path.join(directory, filename)
#     annotated_image = annotate_image(image_path, data)
#     output_path = 'google_ocr_temp/annotated_' + filename
#     annotated_image.save(output_path)
#     print(f"Saved annotated image: {output_path}")


Saved annotated image: temp/annotated_Invoice-Sample.png
Saved annotated image: temp/annotated_invoice-template-us-us-flag-750px.png
Saved annotated image: temp/annotated_af92c6c63efa44cfb0e290fbf8e2d9c4.png


# anoote for openai results

In [24]:
len(res_sample)

39

In [21]:
res_sample = []

for res in responses:
    try:
        # Clean the response and replace 'null' with 'None', then evaluate it
        cleaned_response = res["response"].replace("\n", "").replace("```json", "").replace("```", "").replace("null", "None")
        result = eval(cleaned_response)
        res_sample.append(result)
    except SyntaxError as e:
        print(res)
        print("--------------------")
        print(f"Error evaluating response: {e}")

# Now 'res_sample' contains the structured JSON data, and any errors are handled gracefully
res_sample


{'key': 'creating-invoices-on-excel.png', 'response': 'Based on the provided OCR data, here is the structured JSON output extracting the relevant invoice information:\n\n```json\n{\n  "invoice_number": "1000001",\n  "invoice_number_bbox": "[772, 198, 824, 210]",\n  "issue_date": "July 6, 2012",\n  "issue_date_bbox": "[771, 175, 854, 189]",\n  "total": "$2,407.00",\n  "total_bbox": "[897, 1002, 955, 1014]",\n  "table": "QUANTITY DESCRIPTION UNIT PRICE AMOUNT\\n120 Widgets $10.00 $1,200.00\\n1,000 Tin Badges $1.00 $1,000.00",\n  "table_bbox": "[18, 427, 956, 506]"\n}\n```\n\nPlease note that the \'table\' field in the JSON is an approximation of the layout based on available data, and the \'table_bbox\' encompasses the entire area covering the list of items, unit prices, and amounts. The other fields (\'invoice_number\', \'issue_date\', and \'total\') and their bounding boxes have been extracted based on the recognized patterns in the OCR data that are indicative of those specific invoic

[{'invoice_number': '30037',
  'invoice_number_bbox': '[1492, 177, 1570, 197]',
  'issue_date': 'DEC03',
  'issue_date_bbox': '[1487, 242, 1572, 263]',
  'total': '8,385.00',
  'total_bbox': '[882, 1809, 962, 1828]',
  'table': 'Item Description Qty. Unit Price Ext. Price\nAEROCHAIR - E1 Aero Executive Tile Chair 5 $500.00 $2,500.00\nCHAIR - SW4 Office Chair - Swivel Grey 4 Units $478.00 $1,912.00\nCHAIR - SW4 Office Chair - Swivel Grey 5 $699.00 $3,495.00',
  'table_bbox': '[451, 960, 1670, 1730]'},
 {'invoice_number': '123456789WDF',
  'invoice_number_bbox': '[196, 1701, 440, 1728]',
  'issue_date': '14 Sept 2017',
  'issue_date_bbox': '[1367, 1702, 1563, 1730]',
  'total': '£918.00',
  'total_bbox': '[2197, 3010, 2308, 3038]',
  'table': 'Item Description Qty. Total\nSamsung Galaxy Note 8 (SM-N950FZKABTU), 64GB, Black 1 £724.17 £765.00\nSamsung Galaxy Note 8 Clear View Standing Cover (EF-ZN950CNEGWW), Deep Blue 1 £40.83 £153.00',
  'table_bbox': '[667, 2253, 1079, 2430]'},
 {'invoic

In [23]:
for res, sam in zip(responses, res_sample):
    sam["title"] = res["key"]

In [25]:
res_sample

[{'invoice_number': '30037',
  'invoice_number_bbox': '[1492, 177, 1570, 197]',
  'issue_date': 'DEC03',
  'issue_date_bbox': '[1487, 242, 1572, 263]',
  'total': '8,385.00',
  'total_bbox': '[882, 1809, 962, 1828]',
  'table': 'Item Description Qty. Unit Price Ext. Price\nAEROCHAIR - E1 Aero Executive Tile Chair 5 $500.00 $2,500.00\nCHAIR - SW4 Office Chair - Swivel Grey 4 Units $478.00 $1,912.00\nCHAIR - SW4 Office Chair - Swivel Grey 5 $699.00 $3,495.00',
  'table_bbox': '[451, 960, 1670, 1730]',
  'title': '01AR_print_inv.jpg'},
 {'invoice_number': '123456789WDF',
  'invoice_number_bbox': '[196, 1701, 440, 1728]',
  'issue_date': '14 Sept 2017',
  'issue_date_bbox': '[1367, 1702, 1563, 1730]',
  'total': '£918.00',
  'total_bbox': '[2197, 3010, 2308, 3038]',
  'table': 'Item Description Qty. Total\nSamsung Galaxy Note 8 (SM-N950FZKABTU), 64GB, Black 1 £724.17 £765.00\nSamsung Galaxy Note 8 Clear View Standing Cover (EF-ZN950CNEGWW), Deep Blue 1 £40.83 £153.00',
  'table_bbox': '[66

In [26]:
res_sample.append({
  "title": "e1a0ba8226a03551145610bd34968cc5.jpg",
  "invoice_number": "500117A",
  "invoice_number_bbox": "[691, 171, 714, 175]",
  "issue_date": "09/06/2012",
  "issue_date_bbox": "[403, 159, 460, 169]",
  "total": "$39.65",
  "total_bbox": "[626, 143, 638, 159]",
  "table": "SERVICE DATE OF SERVICE Billed charges to date Receipts to date Adjustments to date Insurance Pending Balance Forward 07/19/12 PAYMENT CHECK; 1313 Billed charges to date Receipts to date Adjustments to date Insurance Pending Account Balance",
  "table_bbox": "[29, 395, 697, 499]"
})

In [27]:
res_sample.append(
    {
      "title": 'creating-invoices-on-excel.png',
  "invoice_number": "1000001",
  "invoice_number_bbox": "[772, 198, 824, 210]",
  "issue_date": "July 6, 2012",
  "issue_date_bbox": "[771, 175, 854, 189]",
  "total": "$2,407.00",
  "total_bbox": "[897, 1002, 955, 1014]",
  "table": "QUANTITY DESCRIPTION UNIT PRICE AMOUNT\n120 Widgets $10.00 $1,200.00\n1,000 Tin Badges $1.00 $1,000.00",
  "table_bbox": "[18, 427, 956, 506]"
}
)

In [28]:
len(res_sample)

41

In [29]:
import ast  # Safe alternative to eval


padding = 8
extra_padding = 175  # Extra padding for table_bbox

def apply_padding(bbox, field):
    if field == 'table_bbox':
        return [abs(bbox[0] - extra_padding), bbox[1] - padding, bbox[2] + extra_padding, bbox[3] + padding]
    else:
        return [bbox[0] - padding, bbox[1] - padding, bbox[2] + padding, bbox[3] + padding]

scaled_data = []

for entry in res_sample:
    scaled_entry = entry.copy()  # Copy the original entry
    for field in ['invoice_number_bbox', 'issue_date_bbox', 'total_bbox', 'table_bbox']:
        bbox = ast.literal_eval(entry[field])  # Convert string representation of list to actual list
        scaled_entry[field] = str(apply_padding(bbox, field))  # Store the scaled bbox back as a string
    scaled_data.append(scaled_entry)

# Now 'scaled_data' contains the updated entries with scaled bounding boxes


ValueError: malformed node or string: None

In [47]:
scaled_data

[{'invoice_number': '1500-1598',
  'invoice_number_bbox': '[69, 251, 142, 277]',
  'issue_date': '24th June 2020',
  'issue_date_bbox': '[562, 254, 660, 280]',
  'total': '$18.00',
  'total_bbox': '[585, 463, 638, 487]',
  'table': 'Product Qty. Price\nBeanie 1 $18.00',
  'table_bbox': '[95, 364, 805, 389]',
  'title': 'Invoice-Sample.png'},
 {'invoice_number': 'US-001',
  'invoice_number_bbox': '[667, 206, 702, 231]',
  'issue_date': '11/02/2019',
  'issue_date_bbox': '[626, 230, 702, 258]',
  'total': '$154.06',
  'total_bbox': '[629, 555, 691, 587]',
  'table': 'DESCRIPTION Qty UNIT PRICE\nFront and rear brake cables 1 100.00\nNew set of pedal arms 2 15.00\nLabor 3hrs 3 5.00',
  'table_bbox': '[48, 347, 696, 475]',
  'title': 'invoice-template-us-us-flag-750px.png'}]

In [30]:
final_data = pd.DataFrame(res_sample)
final_data

,invoice_number,invoice_number_bbox,issue_date,issue_date_bbox,total,total_bbox,table,table_bbox,title
0,30037,"[1492, 177, 1570, 197]",DEC03,"[1487, 242, 1572, 263]","8,385.00","[882, 1809, 962, 1828]",Item Description Qty. Unit Price Ext. Price\nA...,"[451, 960, 1670, 1730]",01AR_print_inv.jpg
1,123456789WDF,"[196, 1701, 440, 1728]",14 Sept 2017,"[1367, 1702, 1563, 1730]",£918.00,"[2197, 3010, 2308, 3038]",Item Description Qty. Total\nSamsung Galaxy No...,"[667, 2253, 1079, 2430]",creating-invoices-on-excel.png
2,00121,"[659, 77, 696, 89]","April 8, 2015","[45, 293, 124, 306]",$270.00,"[637, 605, 691, 618]",Description Qty Unit Price Total\nBrochure Des...,"[55, 347, 569, 422]",Samsung Invoice.jpg
3,CT000000001,"[319, 129, 408, 138]","October 10, 2020","[40, 129, 147, 138]","€1,20,000.00","[609, 703, 684, 712]",ITEM NAME QTY PRICE / DISCOUNT / GST AMOUNT\nB...,"[45, 498, 685, 608]",e1a0ba8226a03551145610bd34968cc5.jpg
4,SOtt306386,"[1671, 526, 1803, 542]",None,None,"$4,763.45","[2155, 2413, 2244, 2431]",WOVEN BACKPACK 100 % NYLON 10 $ 48.11\nWOVEN T...,"[297, 1039, 2006, 1686]",excel-ss.png
5,567,"[523, 242, 529, 252]",7/9/2015,"[686, 242, 736, 252]",$349.85,"[742, 572, 780, 581]",Description Quantity Rate Amount\nPilot Traini...,"[53, 406, 780, 543]",image-414 (1).png
6,204,"[29, 51, 45, 61]",19-Nov-20,"[272, 99, 319, 108]","€ 62,000.00","[434, 476, 486, 486]",SI No. Description of Goods Quantity Rate per ...,"[19, 273, 485, 313]",Commercial-Invoice-2.1_1_ysej-7r.jpg
7,S1405G202003283,"[454, 1007, 597, 1020]",2020-03-07,"[630, 535, 699, 545]",10816.98,"[712, 1931, 769, 1940]",Parts / OP code 1 Labour and Services 5 HYUNDA...,"[127, 746, 803, 1905]",sample-invoice.png
8,38,"[251, 48, 263, 55]",19-Nov-20,"[369, 48, 417, 56]","€7,000.00","[435, 624, 481, 633]",Particulars Repairs and Maintenance Charges\nQ...,"[54, 227, 481, 237]",delivery-note.jpg
9,CODE0001,"[494, 246, 545, 255]",09/27/2018,"[406, 368, 459, 376]",200.00 USD,"[735, 665, 768, 672]",DESCRIPTION QTY WEIGHT RATE AMOUNT\nINLAND FRE...,"[351, 622, 768, 672]",Service.jpg


# Anoate the openai data

In [31]:
from PIL import Image, ImageDraw
import ast  # Safe alternative to eval

def annotate_image(image_path, annotation):
    # Open the image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # Define the fields that contain bounding box data
    bbox_fields = ['invoice_number_bbox', 'issue_date_bbox', 'total_bbox', 'table_bbox']

    for field in bbox_fields:
        if field in annotation and annotation[field]:
            # Extract text and bbox from the annotation
            text = annotation[field.replace('_bbox', '')]
            try:
                bbox = ast.literal_eval(annotation[field])  # Safely convert string to list
            except ValueError:
                print(f"Could not parse bbox for {field} in {image_path}")
                continue
            
            draw.rectangle([bbox[0]  , bbox[1] , bbox[2]  , bbox[3] ], outline='red', width=2)
    
            # Draw the text slightly above the top-left corner of the bounding box
            # draw.text((bbox[0], bbox[1] - 15), text, fill='black')  # Adjust y-offset as needed

    return image




In [32]:

# Annotate each image and save
for annotation in res_sample:
    filename = annotation['title']
    image_path = os.path.join(directory, filename)  # Define 'directory' where images are stored
    annotated_image = annotate_image(image_path, annotation)
    output_path = 'openai_temp_test/annotated_' + filename
    annotated_image.save(output_path)
    print(f"Saved annotated image: {output_path}")

Saved annotated image: openai_temp_test/annotated_01AR_print_inv.jpg
Saved annotated image: openai_temp_test/annotated_creating-invoices-on-excel.png
Saved annotated image: openai_temp_test/annotated_Samsung Invoice.jpg
Saved annotated image: openai_temp_test/annotated_e1a0ba8226a03551145610bd34968cc5.jpg
Saved annotated image: openai_temp_test/annotated_excel-ss.png
Saved annotated image: openai_temp_test/annotated_image-414 (1).png
Saved annotated image: openai_temp_test/annotated_Commercial-Invoice-2.1_1_ysej-7r.jpg
Saved annotated image: openai_temp_test/annotated_sample-invoice.png
Saved annotated image: openai_temp_test/annotated_delivery-note.jpg
Saved annotated image: openai_temp_test/annotated_Service.jpg
Saved annotated image: openai_temp_test/annotated_service-invoice.jpg
Saved annotated image: openai_temp_test/annotated_commercial-invoice-template.png
Saved annotated image: openai_temp_test/annotated_Invoice Sample Photography.jpg
Saved annotated image: openai_temp_test/ann

# Annote the train data

In [37]:

# # Annotate each image and save
# for annotation in train.to_dict(orient="records")[0:2]:
#     filename = annotation['title']
#     image_path = os.path.join(directory, filename)  # Define 'directory' where images are stored
#     annotated_image = annotate_image(image_path, annotation)
#     output_path = 'openai_temp/annotated_train_' + filename
#     annotated_image.save(output_path)
#     print(f"Saved annotated image: {output_path}")


In [82]:
scaled_final_data = pd.DataFrame(scaled_data)
scaled_final_data

,invoice_number,invoice_number_bbox,issue_date,issue_date_bbox,total,total_bbox,table,table_bbox,title
0,1500-1598,"[69, 251, 142, 277]",24th June 2020,"[562, 254, 660, 280]",$18.00,"[585, 463, 638, 487]",Beanie with Logo 1 $18.00,"[-96, 335, 389, 393]",Invoice-Sample.png
1,US-001,"[667, 206, 702, 231]",11/02/2019,"[626, 230, 702, 258]",$154.06,"[629, 555, 691, 587]",DESCRIPTION QTY UNIT PRICE\nFront and rear bra...,"[50, 347, 696, 475]",invoice-template-us-us-flag-750px.png


In [84]:
# Now 'scaled_final_data' contains the updated bounding boxes
# Save the updated DataFrame
scaled_final_data.to_csv("llm_final_data.csv", index=False)

print("Saved scaled data to 'scaled_final_data.csv'")

Saved scaled data to 'scaled_final_data.csv'


In [34]:
final_data.to_csv("llm_final_data.csv", index=False)

In [35]:
final_data

,invoice_number,invoice_number_bbox,issue_date,issue_date_bbox,total,total_bbox,table,table_bbox,title
0,30037,"[1492, 177, 1570, 197]",DEC03,"[1487, 242, 1572, 263]","8,385.00","[882, 1809, 962, 1828]",Item Description Qty. Unit Price Ext. Price\nA...,"[451, 960, 1670, 1730]",01AR_print_inv.jpg
1,123456789WDF,"[196, 1701, 440, 1728]",14 Sept 2017,"[1367, 1702, 1563, 1730]",£918.00,"[2197, 3010, 2308, 3038]",Item Description Qty. Total\nSamsung Galaxy No...,"[667, 2253, 1079, 2430]",creating-invoices-on-excel.png
2,00121,"[659, 77, 696, 89]","April 8, 2015","[45, 293, 124, 306]",$270.00,"[637, 605, 691, 618]",Description Qty Unit Price Total\nBrochure Des...,"[55, 347, 569, 422]",Samsung Invoice.jpg
3,CT000000001,"[319, 129, 408, 138]","October 10, 2020","[40, 129, 147, 138]","€1,20,000.00","[609, 703, 684, 712]",ITEM NAME QTY PRICE / DISCOUNT / GST AMOUNT\nB...,"[45, 498, 685, 608]",e1a0ba8226a03551145610bd34968cc5.jpg
4,SOtt306386,"[1671, 526, 1803, 542]",None,None,"$4,763.45","[2155, 2413, 2244, 2431]",WOVEN BACKPACK 100 % NYLON 10 $ 48.11\nWOVEN T...,"[297, 1039, 2006, 1686]",excel-ss.png
5,567,"[523, 242, 529, 252]",7/9/2015,"[686, 242, 736, 252]",$349.85,"[742, 572, 780, 581]",Description Quantity Rate Amount\nPilot Traini...,"[53, 406, 780, 543]",image-414 (1).png
6,204,"[29, 51, 45, 61]",19-Nov-20,"[272, 99, 319, 108]","€ 62,000.00","[434, 476, 486, 486]",SI No. Description of Goods Quantity Rate per ...,"[19, 273, 485, 313]",Commercial-Invoice-2.1_1_ysej-7r.jpg
7,S1405G202003283,"[454, 1007, 597, 1020]",2020-03-07,"[630, 535, 699, 545]",10816.98,"[712, 1931, 769, 1940]",Parts / OP code 1 Labour and Services 5 HYUNDA...,"[127, 746, 803, 1905]",sample-invoice.png
8,38,"[251, 48, 263, 55]",19-Nov-20,"[369, 48, 417, 56]","€7,000.00","[435, 624, 481, 633]",Particulars Repairs and Maintenance Charges\nQ...,"[54, 227, 481, 237]",delivery-note.jpg
9,CODE0001,"[494, 246, 545, 255]",09/27/2018,"[406, 368, 459, 376]",200.00 USD,"[735, 665, 768, 672]",DESCRIPTION QTY WEIGHT RATE AMOUNT\nINLAND FRE...,"[351, 622, 768, 672]",Service.jpg
